在这个notebook中，我们展示了使用PINNsFormer方法求解ns方程的示例。

在训练结束后，可视化了pinnsformer方法求解和精确解的误差。

这个notebook中的代码本质是navier_stokes目录中代码的梳理，如果需要更细致的对比研究，请在四个对应的目录中运行相关的notebook文件。

详细的项目介绍请参考REAEDME.md文件。

#### 1.模型介绍
PINNsFormer是一种基于transformer的物理信息神经网络(PINN)，PINN结合了物理信息与神经网络，可以用于求解非线性偏微分方程(PDEs)。然而，传统的基于多层感知器（MLP）的PINNs忽视了实际物理系统中固有的重要时间依赖性，因此无法全局传播初始条件约束，并在不同场景下准确捕捉真实解。PINNsFormer通过利用多头注意力机制来捕捉时间依赖性，从而能够更准确地近似PDE解。PINNsFormer将点式输入转换为伪序列，并用序列损失取代点式PINNs损失。

#### 2.环境安装
本项目环境基于Python 3.10，DTK 24.04，PyTorch 2.0, OneScience, json, pickle, getopt等依赖库。
在超算互联网平台上，我们提供了预先配置好的镜像，创建容器后即可获得已经设置好的运行环境，实现一键运行。
如果onescience更新，需要手动安装onescience， onescience的包位于deepcfd工程文件中。
可以使用以下指令手动更新安装onescience。

In [ ]:
#请在python环境中检查，如果环境没有更新onescience，取消注释下一行代码并执行安装onescience
!pip install onescience-0.1.1-py3-none-any.whl

#### 3.环境依赖检查
环境检测包含pytorch版本和dcu环境检测。

In [ ]:
import torch
version = torch.__version__
num = float(version[:3])
assert num >= 2.0, "Pytorch version must >= 2.0"

assert torch.cuda.is_available(), "Pytorch need DCU"

#### 4.素材准备
本例的数据位于navier_stokes目录下./navier_stokes/cylinder_nektar_wake.mat。这是一组圆柱绕流的仿真数据。

#### 5.训练模型
这部分代码包括模型定义，数据集定义和训练过程。

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import random
from torch.optim import LBFGS, Adam
from tqdm import tqdm
import scipy.io
import sys
import os
from torch.utils.data import Dataset,DataLoader

from onescience.utils.pinnsformer_util import *

seed = 0
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

data = scipy.io.loadmat('./navier_stokes/cylinder_nektar_wake.mat')

U_star = data['U_star'] # N x 2 x T
P_star = data['p_star'] # N x T
t_star = data['t'] # T x 1
X_star = data['X_star'] # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

# Rearrange Data 
XX = np.tile(X_star[:,0:1], (1,T)) # N x T
YY = np.tile(X_star[:,1:2], (1,T)) # N x T
TT = np.tile(t_star, (1,N)).T # N x T

UU = U_star[:,0,:] # N x T
VV = U_star[:,1,:] # N x T
PP = P_star # N x T

x = XX.flatten()[:,None] # NT x 1
y = YY.flatten()[:,None] # NT x 1
t = TT.flatten()[:,None] # NT x 1

u = UU.flatten()[:,None] # NT x 1
v = VV.flatten()[:,None] # NT x 1
p = PP.flatten()[:,None] # NT x 1

idx = np.random.choice(N*T,2500, replace=False)
x_train = x[idx,:]
y_train = y[idx,:]
t_train = t[idx,:]
u_train = u[idx,:]
v_train = v[idx,:]

x_train = np.expand_dims(np.tile(x_train[:], (5)) ,-1)
y_train = np.expand_dims(np.tile(y_train[:], (5)) ,-1)
t_train = make_time_sequence(t_train, num_step=5, step=1e-2)

x_train = torch.tensor(x_train, dtype=torch.float32, requires_grad=True).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32, requires_grad=True).to(device)
t_train = torch.tensor(t_train, dtype=torch.float32, requires_grad=True).to(device)
u_train = torch.tensor(u_train, dtype=torch.float32, requires_grad=True).to(device)
v_train = torch.tensor(v_train, dtype=torch.float32, requires_grad=True).to(device)

class MyDataset(Dataset):
    def __init__(self, x, y, t, u, v):
        self.x = x
        self.y = y
        self.t = t
        self.u = u
        self.v = v

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx], self.t[idx], self.u[idx], self.v[idx]
        
dataset = MyDataset(x_train, y_train, t_train, u_train, v_train)
batch_size = 128  
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

class WaveAct(nn.Module):
    def __init__(self):
        super(WaveAct, self).__init__() 
        self.w1 = nn.Parameter(torch.ones(1), requires_grad=True)
        self.w2 = nn.Parameter(torch.ones(1), requires_grad=True)

    def forward(self, x):
        return self.w1 * torch.sin(x) + self.w2 * torch.cos(x)

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=256):
        super(FeedForward, self).__init__() 
        self.linear = nn.Sequential(*[
            nn.Linear(d_model, d_ff),
            WaveAct(),
            nn.Linear(d_ff, d_ff),
            WaveAct(),
            nn.Linear(d_ff, d_model)
        ])

    def forward(self, x):
        return self.linear(x)


class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()

        self.attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=heads, batch_first=True)
        self.ff = FeedForward(d_model)
        self.act1 = WaveAct()
        self.act2 = WaveAct()
        
    def forward(self, x):
        x2 = self.act1(x)
        x = x + self.attn(x2,x2,x2)[0]
        x2 = self.act2(x)
        x = x + self.ff(x2)
        return x


class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads):
        super(DecoderLayer, self).__init__()

        self.attn = nn.MultiheadAttention(embed_dim=d_model, num_heads=heads, batch_first=True)
        self.ff = FeedForward(d_model)
        self.act1 = WaveAct()
        self.act2 = WaveAct()

    def forward(self, x, e_outputs): 
        x2 = self.act1(x)
        x = x + self.attn(x2, e_outputs, e_outputs)[0]
        x2 = self.act2(x)
        x = x + self.ff(x2)
        return x


class Encoder(nn.Module):
    def __init__(self, d_model, N, heads):
        super(Encoder, self).__init__()
        self.N = N
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.act = WaveAct()

    def forward(self, x):
        for i in range(self.N):
            x = self.layers[i](x)
        return self.act(x)


class Decoder(nn.Module):
    def __init__(self, d_model, N, heads):
        super(Decoder, self).__init__()
        self.N = N
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.act = WaveAct()
        
    def forward(self, x, e_outputs):
        for i in range(self.N):
            x = self.layers[i](x, e_outputs)
        return self.act(x)



class PINNsformer(nn.Module):
    def __init__(self, d_out, d_model, d_hidden, N, heads):
        super(PINNsformer, self).__init__()

        self.linear_emb = nn.Linear(3, d_model)

        self.encoder = Encoder(d_model, N, heads)
        self.decoder = Decoder(d_model, N, heads)
        self.linear_out = nn.Sequential(*[
            nn.Linear(d_model, d_hidden),
            WaveAct(),
            nn.Linear(d_hidden, d_hidden),
            WaveAct(),
            nn.Linear(d_hidden, d_out)
        ])

    def forward(self, x, y, t):
        src = torch.cat((x,y,t), dim=-1)
        src = self.linear_emb(src)
        e_outputs = self.encoder(src)
        d_output = self.decoder(src, e_outputs)
        output = self.linear_out(d_output)
        return output
    

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)

model = PINNsformer(d_out=2, d_hidden=512, d_model=32, N=1, heads=2).to(device)
model.apply(init_weights)
optim = LBFGS(model.parameters(), lr=0.01, line_search_fn='strong_wolfe')

n_params = get_n_params(model)

print(model)
print("totol param numbers:",get_n_params(model))

loss_track = []

for epoch in range(10):  # 假设训练 10 个 epoch
    for batch in tqdm(dataloader):
        x_batch, y_batch, t_batch, u_batch, v_batch = batch
        
        def closure():
            psi_and_p = model(x_batch, y_batch, t_batch)
            psi = psi_and_p[:,:,0:1]
            p = psi_and_p[:,:,1:2]

            u = torch.autograd.grad(psi, y_batch, grad_outputs=torch.ones_like(psi), retain_graph=True, create_graph=True)[0]
            v = - torch.autograd.grad(psi, x_batch, grad_outputs=torch.ones_like(psi), retain_graph=True, create_graph=True)[0]

            u_t = torch.autograd.grad(u, t_batch, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
            u_x = torch.autograd.grad(u, x_batch, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
            u_y = torch.autograd.grad(u, y_batch, grad_outputs=torch.ones_like(u), retain_graph=True, create_graph=True)[0]
            u_xx = torch.autograd.grad(u, x_batch, grad_outputs=torch.ones_like(u_x), retain_graph=True, create_graph=True)[0]
            u_yy = torch.autograd.grad(u, y_batch, grad_outputs=torch.ones_like(u_y), retain_graph=True, create_graph=True)[0]

            v_t = torch.autograd.grad(v, t_batch, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
            v_x = torch.autograd.grad(v, x_batch, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
            v_y = torch.autograd.grad(v, y_batch, grad_outputs=torch.ones_like(v), retain_graph=True, create_graph=True)[0]
            v_xx = torch.autograd.grad(v, x_batch, grad_outputs=torch.ones_like(v_x), retain_graph=True, create_graph=True)[0]
            v_yy = torch.autograd.grad(v, y_batch, grad_outputs=torch.ones_like(v_y), retain_graph=True, create_graph=True)[0]

            p_x = torch.autograd.grad(p, x_batch, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0]
            p_y = torch.autograd.grad(p, y_batch, grad_outputs=torch.ones_like(p), retain_graph=True, create_graph=True)[0]

            f_u = u_t + (u * u_x + v * u_y) + p_x - 0.01 * (u_xx + u_yy) 
            f_v = v_t + (u * v_x + v * v_y) + p_y - 0.01 * (v_xx + v_yy)

            loss = (
                torch.mean((u[:, 0] - u_batch) ** 2) +
                torch.mean((v[:, 0] - v_batch) ** 2) +
                torch.mean(f_u ** 2) +
                torch.mean(f_v ** 2)
            )

            loss_track.append(loss.item())

            optim.zero_grad()
            loss.backward()
            return loss

        optim.step(closure)
 
save_flag = False
if save_flag:        
    np.save('./ns_loss_pinnsformer.npy', loss_track)
    torch.save(model.state_dict(), './ns_pinnsformer.pt')

print("loss after train:",loss_track[-1])

#### 5.测试
对训练好的模型进行评估。

In [ ]:
snap = np.array([100])
x_star = X_star[:,0:1]
y_star = X_star[:,1:2]
t_star = TT[:,snap]

u_star = U_star[:,0,snap]
v_star = U_star[:,1,snap]
p_star = P_star[:,snap]

x_star = np.expand_dims(np.tile(x_star[:], (5)) ,-1)
y_star = np.expand_dims(np.tile(y_star[:], (5)) ,-1)
t_star = make_time_sequence(t_star, num_step=5, step=1e-2)

x_star = torch.tensor(x_star, dtype=torch.float32, requires_grad=True).to(device)
y_star = torch.tensor(y_star, dtype=torch.float32, requires_grad=True).to(device)
t_star = torch.tensor(t_star, dtype=torch.float32, requires_grad=True).to(device)

psi_and_p = model(x_star, y_star, t_star)
psi = psi_and_p[:,:,0:1]
p_pred = psi_and_p[:,:,1:2]

u_pred = torch.autograd.grad(psi, x_star, grad_outputs=torch.ones_like(psi), retain_graph=True, create_graph=True)[0]
v_pred = - torch.autograd.grad(psi, y_star, grad_outputs=torch.ones_like(psi), retain_graph=True, create_graph=True)[0]

u_pred = u_pred.cpu().detach().numpy()[:,0]
v_pred = v_pred.cpu().detach().numpy()[:,0]
p_pred = p_pred.cpu().detach().numpy()[:,0]


error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_p = np.linalg.norm(p_star-p_pred,2)/np.linalg.norm(p_star,2)

#### 6.可视化
模型求解值（推理结果）和理论求解值（标签）以及两者误差的可视化。

In [ ]:
plt.figure(figsize=(4,3))
plt.imshow((p_star).reshape(50,100), extent=[-3,8,-2,2], aspect='auto')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Exact p(x,t)')
plt.colorbar()
plt.tight_layout()
if save_flag:
    plt.savefig('./ns_exact.png')
plt.show()

In [ ]:
plt.figure(figsize=(4,3))
plt.imshow((p_pred).reshape(50,100), extent=[-3,8,-2,2], aspect='auto')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Predicted p(x,t)')
plt.colorbar()
plt.tight_layout()
if save_flag:
    plt.savefig('./ns_pinnsformer_pred.png')
plt.show()

In [ ]:
plt.figure(figsize=(4,3))
plt.imshow(np.abs(p_pred-p_star).reshape(50,100), extent=[-3,8,-2,2], aspect='auto')
plt.xlabel('x')
plt.ylabel('y')
plt.title('Absolute Error')
plt.colorbar()
plt.tight_layout()
if save_flag:
    plt.savefig('./ns_pinnsformer_error.png')
plt.show()